# Introduction to machine learning with TensorFlow

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Common imports
import fitsio
import numpy as np
import os
import pandas as pd
import sys

In [ ]:
# To make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.compat.v1.reset_default_graph
    tf.random.set_seed(seed)
    np.random.seed(seed)

In [ ]:
# To plot pretty figures
import matplotlib
import matplotlib.pyplot as plt
plt.rc('font', size=18)

In [ ]:
# Tensorflow imports
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [ ]:
# QuasarNET imports
from quasarnet.models import QuasarNET, custom_loss
from quasarnet.io import read_truth, read_data, objective, wave
from quasarnet.utils import process_preds, absorber_IGM

## 1. Machine learning in a nutshell

The search for artificial intelligence has brought two key points to light:

 - Computational tasks that are difficult for humans are often easy for machines (e.g. multiplication of large matrices)
 - Intuitive tasks that are easy for humans are often difficulty for machines (e.g. image recognition)
 
Machine learning seeks to address this second point by constructing a learning process by which a machine can gain experience. 

Rather than defining a rigid set of rules, we instead define a framework of some sort, and allow the machine to learn the rules within this framework.

### Learning types

ML is generally grouped into 3 main categories which have different aims

1. **Supervised learning**: learn to predict a given output quantity, having learned from many "labelled" examples
2. **Unsupervised learning**: learn to find structures and representations of the input data
3. **Reinforcement learning**: learn a certain action to maximise a payoff

Each of these is a huge topic, so we'll only consider supervised learning for now. Within supervised learning, problems generally fall into one of two types:
1. **Regression**: predict an output number (e.g. determine redshift)
2. **Classification**: predict which class data belongs to (e.g. classify a spectrum as star/galaxy/quasar)

### Key terms

In general, we can describe the approach of supervised machine learning as seeking to learn a mapping from some input data $X$ to some output data $Y$:

$$ Y = f(X; \theta), $$

where $f$ is the model ("framework") that we choose, and $\theta$ is a vector containing the parameters of this model. It is the values in $\theta$ that we seek to learn.

Typically, $X$ and $Y$ may be vectors, and we describe these with the terms:

 - **Feature**: an element of the input data $X$. If the input data is an image, then the features may be its pixels. 
 - **Target**: the output that we are trying to predict, $Y$, such as the value of redshift, or an object's class.

When looking for the optimal values of $\theta$, we seek to minimise a cost (or loss) function $C(\theta)$, which describes how well a given set of parameters describes the mapping from features to target output.

### A simple example: linear regression

Given some some input data, we would like to try and predict a certain quantity by finding a simple, linear, "best-fit" model.

In other words: for a dependent variable $Y$ and independent variables $\mathbf{X}=(X_0, X_1, ... , X_n)$, our model is:

$$ Y = \theta_0 + \theta_1 X_1 + \theta_2 X_2 + ... + \theta_n X_n,$$

where the $\theta_j$ are the parameters of our model. $\theta_0$ is known as the "bias" - it is essentially an intercept term.

In vector form, we can write this as:

$$Y = \mathbf{\theta}^T \mathbf{X},$$

where $\mathbf{\theta}=(\theta_0,\theta_1,\theta_2,...,\theta_n)^T$, and $\cdot^T$ denotes the transpose.

We may measure $m$ data points, giving us our measured values of the dependent variable $y^{(i)}$ and measured values of the independent variables $x^{(i)}_j$. Given these measurements, we then want to deduce what are the best values of $\theta^{(j)}$.

Typically, we will do this by minimising the mean squared error (MSE):

$$\text{MSE} = \frac{1}{m} \sum_{i=1}^{m} \left(\theta^{\rm T} x^{(i)} - y^{(i)}\right)^2 .$$

Minimising the MSE is equivalent to minimising the cost function

$$C(\theta) = \frac{1}{m} (x \theta - y)^{\rm T}(x \theta - y),$$

where for notational convenience we denote the dependence on $\theta$ only and consider $x$ and $y$ fixed.

#### Analytic solution

We may solve this problem analytically with a bit of linear algebra. This gives us a solution:

$$ \hat{\theta} = \left( X^{\rm T} X \right)^{-1} X^{\rm T} y. $$

Let's consider a very simple example. First we'll generate some (noisy) data:

In [ ]:
# Number of data points
m = 100

# y-intercept and slope of line
theta_0 = 4
theta_1 = 3

# Noise about true value
sigma_random = 1

# Random x values from 0 to 2, y values with noise around true line
X_lr = 2 * np.random.rand(m, 1)
y_lr = theta_0 + theta_1 * X_lr + np.random.normal(size=(m, 1),scale=sigma_random)

plt.figure(figsize=(9,6))
plt.plot(X_lr, y_lr, "b.")
plt.xlabel("$x_1$")
plt.ylabel("$y$")
plt.axis([0, 2, 0, 15]);

And now let's compute the analytic solution for this data:

In [ ]:
# Add the "bias" term to our x values
X_b = np.concatenate((np.ones((m,1)),X_lr),axis=1)

# Compute the analytic solution
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y_lr)

print('Analytic solution:\ntheta_0={:1.3f}, theta_1={:1.3f}'.format(theta_best[0][0],theta_best[1][0]))
print('\nTrue values:\ntheta_0={:1.3f}, theta_1={:1.3f}'.format(theta_0,theta_1))

This seems to have worked quite well. The errors are due to the randomness in the data: try varying the value of `sigma_random` or the number of data points `m` and see what effects this has.

Plotting the best fit line that we have calculated:

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(X_lr, y_lr, "b.")
plt.plot(X_lr, theta_0 + theta_1*X_lr, "grey", label='True')
plt.plot(X_lr, theta_best[0][0] + theta_best[1][0]*X_lr, "r", label='Best fit')
plt.xlabel("$x_1$")
plt.ylabel("$y$")
plt.legend()
plt.axis([0, 2, 0, 15])
plt.show()

#### An alternative approach: gradient descent

The analytic approach works nicely in this case where we have a small number of parameters and a small number of data points. However, it required us to invert $X^TX$, which is an $n\times n$ matrix (recall, $n$ is the number of parameters). 

The computational complexity of this task is of $\mathcal{O}(n^3)$, and typically requires all of our training data to be held in memory at once. When dealing with large datasets, an alternative approach is necessary.

Most commonly, gradient descent is used to find the minimum of the cost function. This algorithm is defined by choosing a random set of starting parameters, and then "descending" towards the minimum of the cost function in a series of discrete, iterated steps. Formally, our iterated parameter valeus are defined by:

$$\theta^{(t)} = \theta^{(t-1)} - \alpha  \frac{\partial C}{\partial \theta},$$

where $t$ denotes the iteration number, alpha is known as the "learning rate" and:

$$ \frac{\partial C}{\partial \theta} 
=\frac{2}{m} X^{\rm T}  \left( X \theta - y \right) 
= \left[\frac{\partial C}{\partial \theta_0},\ \frac{\partial C}{\partial \theta_1},\ ...,\ \frac{\partial C}{\partial \theta_n} \right]^{\rm T} 
.$$

We can visualise this nicely:

<left><img src="Images/optimization_gd.png" style="height: 350px;"/></left>

[[Image source](http://www.holehouse.org/mlclass)]

Implementing a simple gradient descent process for our toy example is quite simple. We consider $\alpha=0.1$, and take 1000 steps starting from $\theta^T=[1,1]$:

In [ ]:
# Define our alpha, number of steps and starting value of theta.
alpha = 0.1
n_steps = 200
theta = np.array(([1.],[1.]))

# Set up a list to store the values of theta.
thetas = np.zeros((n_steps+1,2))
thetas[0,:] = theta.flatten()

# For each step:
for i in range(n_steps):
    
    # Calculate the gradient at this point.
    grad = (2/m)*X_b.T.dot(X_b.dot(theta)-y_lr)
    
    # Modify the current value of theta according to the gradient and the learning rate.
    theta += -alpha*grad
    
    # Add the new value of theta to our list
    thetas[i+1,:] = theta.flatten()
    
print('Gradient descent:\ntheta_0={:1.3f}, theta_1={:1.3f}'.format(theta[0][0],theta[1][0]))

Recall the answers from earlier:

In [ ]:
print('Analytic solution:\ntheta_0={:1.3f}, theta_1={:1.3f}'.format(theta_best[0][0],theta_best[1][0]))
print('\nTrue values:\ntheta_0={:1.3f}, theta_1={:1.3f}'.format(theta_0,theta_1))

It seems that the gradient descent mtehod matches the analytic approach nicely, very reassuring!

#### Choice of $\alpha$

Note: we arbitrarily chose a value of $\alpha=0.1$. In this case, this was easy to see as a "sensible" option as our data is very simple.

If we had chosen a much smaller value, then we may not have reached the minimum of the cost function in our 1000 steps. Equally, if we had chosen a much larger value, we would have jumped around the parameter space and might not have been able to find the minimum either. 

Try varying the value of alpha, and see how the values of theta vary with the number of steps. At what point do you think things will start to go wrong?

#### Evolution of our parameters

Below are plots showing the evolution of $\theta_0$ and $\theta_1$ with the number of steps, and the path that our gradient descent process follows in parameter space. See how they vary as we change alpha.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12,6), sharex=True, sharey=True)

axs[0].plot(range(n_steps+1),thetas[:,0],label='Grad. Desc.')
axs[0].axhline(y=theta_0,c='grey', label='Truth')
axs[0].axhline(y=theta_best[0][0],c='r', label='Analytic')
axs[0].set_xlabel(r'$n_{\mathrm{steps}}$')
axs[0].set_ylabel(r'$\theta_0$')

axs[1].plot(range(n_steps+1),thetas[:,1])
axs[1].axhline(y=theta_1,c='grey')
axs[1].axhline(y=theta_best[1][0],c='r')
axs[1].set_xlabel(r'$n_{\mathrm{steps}}$')
axs[1].set_ylabel(r'$\theta_1$')

axs[0].legend()
axs[0].set_xlim(0,n_steps)
axs[0].set_ylim(0,5)

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,6))

ax.plot(thetas[:,0],thetas[:,1],".-",label='Grad. Desc.',zorder=-1)
ax.scatter(theta_0,theta_1,c='grey',marker='x',s=200, label='Truth')
ax.scatter(theta_best[0][0],theta_best[1][0],c='red',marker='x',s=200, label='Analytic')

ax.set_xlim(0,5)
ax.set_ylim(0,5)
ax.set_xlabel(r'$\theta_0$')
ax.set_ylabel(r'$\theta_1$')
ax.legend(loc=2)

$\alpha$ is an example of a hyperparameter. Finding optimal hyperparameter values is important for ensuring that our processes converge sensibly.

## 2. TensorFlow

TensorFlow (TF) is an open source library of numerical computation tools developed by Google, which is widely used for machine learning. 

It breaks down any given computation into smaller pieces using computational graphs.

### A computational graph

In this instance, a simple function $f(x,y)$ is broken down by considering:
 - two input variables, $x$ and $y$
 - a constant, 2
 - four operations, two lots of $\times$ and two lots of $+$

<left><img src="Images/computational_graph_simple.png" style="height: 350px;"/></left>

[Credit: Geron]

When writing a program with TF, the user constructs this computational graph. This can be done in many languages, but we will only show Python in this demonstration.

TF then runs the graph using well-optimised C++ code.

### Distributed computation

TF can then take advantage of the computational resources at hand, by distributing the computation of different parts of the graph to different CPUs/GPUs.

<left><img src="Images/computational_graph_hpc.png" style="height: 350px;"/></left>

[Credit: Geron]

This allows TF to scale very easily to using high-performance architectures, allowing us to create machine learning models with very large numbers of parameters.

### Constructing this graph

Let's now construct this graph with TF.

First we import TF and reset our graph (this just sets a random seed to make sure our results are reproducible).

In [ ]:
import tensorflow as tf
reset_graph()

We now define our TF variables, and our function to combine them:

In [ ]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

The objects `x` and `y` are tf.Variable objects

In [ ]:
x

In [ ]:
y

On the other hand `f` is a tf.Tensor object:

In [ ]:
f

In the background, TF has computed the value of `f` already:

In [ ]:
print(f'f={f}')

Note for those with some TF experience: this is a different behaviour to v1.x of TF, in which you would have needed to create a "Session", like:

In [ ]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

As we can see, this now returns an error, as TF is not backwards compatible in this way.

### Linear regression with TensorFlow

Let's go back to the example from earlier: linear regression. We can straightforwardly carry out this process from within TF.

First we make our TF objects, making sure to add a "bias" term to $X$ again.

In [ ]:
X_b = np.concatenate([np.ones_like(X_lr),X_lr],axis=1)
X_tf = tf.constant(X_b, dtype=tf.float32, name="X")
y_tf = tf.constant(y_lr, dtype=tf.float32, name="y")

Here, I've used `tf.constant` for our values of $X$ and $y$ as they will not vary.

Choose initial value of theta, the number of steps and the learning rate.

In [ ]:
alpha = 0.1
n_steps = 200
theta_init = np.array(([1.],[1.]))

We now make our variable for $\theta$. Here I use `tf.Variable` as we will want to update this at every step of our gradient descent.

In [ ]:
theta_tf = tf.Variable(theta_init, dtype=tf.float32, name="theta")

Now we carry out gradient descent in the same way that we did before. It's a bit clunky as this isn't really how TF is designed to be used!

In [ ]:
for i in range(n_steps):

    # Calculate the gradient wrt theta
    with tf.GradientTape(persistent=True) as g:
        y_pred = tf.matmul(X_tf, theta_tf, name="predictions")
        error = y_pred - y_lr
        mse = tf.reduce_mean(tf.square(error), name="mse")
        
    grad = g.gradient(mse, theta_tf)
    
    # Update the value of theta
    theta_tf.assign(theta_tf - 0.1*grad)

In [ ]:
print('TensorFlow:\ntheta_0={:1.3f}, theta_1={:1.3f}'.format(theta_tf[0][0],theta_tf[1][0]))

Comparing this to our previous values once again and all looks good:

In [ ]:
print('\nGradient descent:\ntheta_0={:1.3f}, theta_1={:1.3f}'.format(theta[0][0],theta[1][0]))
print('\nAnalytic solution:\ntheta_0={:1.3f}, theta_1={:1.3f}'.format(theta_best[0][0],theta_best[1][0]))
print('\nTrue values:\ntheta_0={:1.3f}, theta_1={:1.3f}'.format(theta_0,theta_1))

### TensorFlow APIs

In the above example, we used TF at a low level. However, as we move to more complex models, manually defining the MSE, gradients etc. will not be feasible. Instead, we want to move to using one of TF's APIs (application programming interfaces). This is a way to make use of the power of TF with only relatively simple commands. 

Historically, there have been many different APIs available, but the current direction is to make use of a sub-package called "Keras". For comparison, below is the code to address the same problem using Keras:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Build the model.
model = Sequential([Dense(units=1, input_shape=(2,), use_bias=False, kernel_initializer=tf.constant_initializer(theta_init))])

# Set up the minimisation to carry out.
model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
             )

# Run the minimisation.
history = model.fit(X_b, y_lr, batch_size=m, epochs=200, verbose=False)

Extracting the parameters from this model, they are similar to those from previous methods.

In [ ]:
theta_keras = model.get_layer(index=0).get_weights()[0]
print('TensorFlow:\ntheta_0={:1.3f}, theta_1={:1.3f}'.format(theta_keras[0][0],theta_keras[1][0]))

This approach uses "Keras Sequential API", the most straightforward way to use Keras. We will later use the "Keras Functional API" which introduces slightly more customisation.

## 3. Neural Networks

TF is a particularly powerful tool for constructing artificial neural networks (ANNs). These are models which took inspiration from the "neural network" that makes up the sensory system of animals. The basic building block of a neural network comes from the neuron. In ANNs, we may represent this as:

<center><img src="Images/general_logistic_unit.png" style="height: 200px;"/></center>

[credit: Jason McEwan]

This takes a set of inputs $x_i$, carries out a weighted sum $z=\sum_{j=1}^n \theta_j x_j =\theta^{\rm T} x$ (orange circle), and then applies an "activation function" $a(z)$. From an input vector $\mathbf{x}$, our neuron outputs $h_\theta(\mathbf{x})$: a parameter-dependent function of our input.

In ML, generally this is referred to as a "logistic unit" rather than a neuron.

The activation function $a$ can take many forms, and careful choice of activation function is important when constructing an ML model. Below are some examples - plot them to compare what they look like and consider why they might be useful:

Linear
$$ a(z) = z $$

Step
$$ a(z) = \left \{
\begin{eqnarray}
0,\ \text{if}\ z < 0\\
1,\ \text{if}\ z \geq 0 \\
\end{eqnarray}
\right. 
$$

Sigmoid
$$
a(z) = \frac{1}{1+\exp{(-z)}}
$$

Hyperboic tangent
$$
a(z) = \tanh(z)
$$

Rectified linear unit (ReLU)
$$
a(z) = \max(0, z)
$$


As a simple example of a neuron, consider the previous "linear regression" problem. In that case:
 - our input data was $\mathbf{x}=(1,x)^T$
 - we carried out a weighted sum $z = \theta_0 \times 1 + \theta_1 \times x$ over the elements of $\mathbf{x}$
 - our output quantity was $y=a(z)=z$
 
In other words, we constructed a single neuron with a linear activation function!

While these neurons are interesting, on their own they are quite limited. However, we can harness great power by building "networks" of many neurons:

<center><img src="Images/ann.png" style="height: 200px;"/></center>

[credit: Jason McEwan]

Furthermore, we can group these into layers which we apply consecutively:

<center><img src="Images/ann_layers.jpg" style="height: 300px;"/></center>

[[Image credit](https://medium.com/@xenonstack/overview-of-artificial-neural-networks-and-its-applications-2525c1addff7)]

As in our simple example, we want to train our neural network via a gradient descent process. This is typically carried out via a process known as "backpropagation" [(Rumelhart, Hinton & Williams, 1986)](https://www.nature.com/articles/323533a0). 

This method efficiently evaluates the gradient of our cost function with respect to each parameter in our network, which we may then make use of when training. There is a nice description of the method [here](http://neuralnetworksanddeeplearning.com/): essentially it relies on using the chain rule, given that we know all of our activation functions.

### In practice

Let's consider a practical example. We'll look at the classic example of the MNIST images. These are a set of hand-written numbers, stored as 28$\times$28 images. We'll first load the data and take a look at some examples

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

Notice that we have split the data into two parts for training and testing. These are effectively two independent sets of $X$ and $y$ values, so that we may train our model with one set, and then test it with the other without worrying that the model might have already "seen" one of the test examples.

In [ ]:
nshow = 3
toshow = np.random.choice(np.arange(x_train.shape[0]), size=nshow, replace=False)

for i in toshow:
    fig, ax = plt.subplots(1,1,figsize=(4,4))
    ax.imshow(x_train[i],cmap='gray')
    ax.set_title('Number {}'.format(y_train[i]))

Our aim is to build a neural network model that can take one of these images as an input, and classify it as being a "0", a "1", ... or a "9".

The process of building a model can be complex, but we will keep it simple for now. In general, there are four very broad questions to consider:
 - how should we process the input data?
 - what is the best architecture? 
 - what is our output?
 - what is the best way to train our model?

#### Processing the data

In general, it is good to try and make sure that our features are all roughly $\mathcal{O}(1)$, and with variation $\mathcal{O}(1)$ as well. This makes the computation of gradients when training our model more straightforward, and ensures that 

In our case, our images are all arrays of size $28\times28$, with values between 0 and 255:

In [ ]:
print('Min =',np.min(x_train),'\nMax =',np.max(x_train))

Thus we may simply divide our data by 255 so that the pixel values vary between 0 and 1.

In [ ]:
x_train = x_train / 255.
x_test = x_test / 255.

#### Model architecture

In general, there are many different types of layers that we may want to include in our neural network. Later, we'll see an example of a convolutional neural network, but for now we'll just use one fully-connected (dense) layer, as we did before.

We may choose the number of units (neurons) in our layer, as well as the activation function. Feel free to play around with these if you like!

In [ ]:
n_units = 128
activation = 'relu'

In [ ]:
layers = [
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(n_units, activation=activation),
]

#### What is our output?

In this example, we would like to classify our images into one of 10 classes. We thus choose our output function to be an array of 10 numbers, containing the probability that our input is in each of our 10 classes. i.e. for our output $Y=(y_1, y_2, ... , y_{10})$:

$$ y_i = P(\mathrm{image\ shows\ digit\ i}) $$

We achieve this by adding a final, fully-connected layer, using a "softmax" activation. This converts a set of numbers into a set of probabilities via the transformation:

$$ 
\hat{p}_j = \frac{\exp(a_j)}{\sum_{j^\prime} \exp(a_{j^\prime})}
.
$$

You can show mathematically that these have been normalised so that
- $\sum_j \hat{p}_j=1$
- $0 \leq \hat{p}_j \leq 1$

i.e. required properties of probabilities. 

In [ ]:
layers += [
  tf.keras.layers.Dense(10, activation='softmax')
]

Our model structure is now complete, and so we make a model within Keras using the layers we have defined:

In [ ]:
model = tf.keras.models.Sequential(layers)

#### How should we train our model?

There are many things to consider when training a model. 
 - How should we feed our input data to the model?
 - What should our cost function be?
 - What algorithm should we follow to find the minimum of the cost function? 
 - What parameters (such as learning rate) should we use?
 - How do we know when to stop training?
 
We could spend a great deal of time going into each of these, and many more questions. 

For now, we will ignore most of these subtleties, considering only the learning rate of our model, and the number of "epochs" of training.

In one "epoch" of training, we pass every training data point through our model, updating the parameters as we go.

Feel free to play with different parameter values below and see how well training works. Equally, feel free to play with the architecture above by adding more layers to see how performance and training time are affected.

We have added the metric of "accuracy" so that we can see how accurate our model's classifications are as it trains (as well as the loss function).

In [ ]:
learning_rate = 0.001
n_epochs = 6

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate),
    metrics=['accuracy'],
)

#### We're ready!

We may now train our model:

In [ ]:
model.fit(
    x_train,
    y_train,
    epochs=n_epochs,
)

Let's make some predictions

In [ ]:
y_pred = np.argmax(model.predict(x_test),axis=1)

In [ ]:
nshow = 3
toshow = np.random.choice(np.arange(x_test.shape[0]), size=nshow, replace=False)

for i in toshow:
    fig, ax = plt.subplots(1,1,figsize=(4,4))
    ax.imshow(x_test[i],cmap='gray')
    
    ax.set_title('Truth {}; prediction {}'.format(y_test[i],y_pred[i]))

We might look to measure the performance of our classifier by its accuracy, i.e. the percentage of classifications it got correct.

In [ ]:
acc = (y_pred == y_test).sum()/len(y_test)
print('Accuracy = {:2.2%}'.format(acc))

## 4. QuasarNET

### Background

In DESI, we will have 4 main samples of objects, the bright galaxy survey (BGS), emission line galaxies (ELGs), luminous red galaxies (LRGs) and quasars (QSOs). For each class of object, we first use imaging data to select targets, which we then observe spectroscopically, giving us a set of observed spectra for each target class.

However, some of our spectra in each target class will be "contaminants". For example, in a set of QSO target spectra, we commonly see a lot of stellar spectra as some stars look similar to QSOs in imaging data. We want to get rid of these contaiminants.

Also, we will need to determine the redshift of every object that we observe, no matter what class it is.

The problems to address then are, for a given spectrum:
 - What class of object is this? (Star, galaxy or QSO)
 - What redshift is it at?
 
Thus we have a joint classification and regression problem.

### Identifying QSOs

We'll focus on QSOs from now on. In early QSO surveys, you would address this problem by "visual inspection": an expert looks at the spectrum and uses their expertise to deduce the class and the redshift. 

For example, below is a spectrum from SDSS. An expert would look at this spectrum, see the broad emission lines and identify this as a QSO. They would then assess which emission line is which, and use the identified emission lines to calculate a redshift. E.g. for the Lyman-$\alpha$ line:

$$ 1+z = \frac{\lambda_\mathrm{obs}}{\lambda_\mathrm{rest}} = \frac{3940}{1215.67} \implies z=2.24 $$

<center><img src="Images/SDSS_QSO_spec.png" style="height: 500px;"/></center>

More recently, it has been common to use an automatic "template fitter", such as redrock. These can construct spectral templates from a set of components, trying to find the best $\chi^2$ fit to the data. They are generally effective and are well-principled, but require detailed understanding of your data and its errors to work well. They also require a comprehensive set of templates in order to be able to classify a wide variety of spectra.

In BOSS DR12, all ~560,000 QSO targets were visually inspected. This was a monumental effort and left us with a large set of very reliable classifications and redshifts. However, it is infeasible to repeat with DESI as there will be too many spectra.

Using the dataset that the DR12 VI effort produced, two machine learning quasar classifiers were developed, one called SQUEzE ([Perez-Rafols et al., 2020](https://arxiv.org/abs/1903.00023), [Perez-Rafols and Pieri, 2020](https://arxiv.org/abs/1911.04891)), and one called QuasarNET ([Busca and Balland, 2018](https://arxiv.org/abs/1808.09955)), which we will focus on now.

### How does QuasarNET work?

QuasarNET attempts to mimic how an expert would identify a QSO:
 - Is it a QSO? -> Does the spectrum have broad emission lines?
 - What is its redshift? -> What wavelength are the broad emission lines at?

It uses a very flexible neural network architecture so that it can try and learn as many subtleties of the data as possible. 

The exact details of how QuasarNET works are a bit beyond the scope of this tutorial\*. We'll go through the same 4 step process as before to show the thinking behind constructing QuasarNET, and then will get onto actually classifying some DESI spectra. 

\*Having said that, it's not **too** complex, so I would encourage anyone keen to take a look at [https://github.com/ngbusca/QuasarNET](https://github.com/ngbusca/QuasarNET).

#### Input data

Spectra from instruments such as BOSS and DESI are generally noisy. When training our classifier to identify QSOs, we want it to pick out **broad** emission lines. This means we can smooth our input spectra quite aggressively to reduce the number of features (pixels) that our model has to deal with.

We can also renormalise our spectra so that they have zero mean and variance 1 (over all wavelengths). This means that there is no (or at least less) dependence on QSO magnitudes.

#### Model architecture

QuasarNET uses several layers in its architecture. Most importantly, it has four "convolutional" layers. These are quite different from the fully-connected layers that we have seen before, using a set of "filters".

In a fully-connected layer, we connect each of our input features to each neuron, then carrying out a weighted sum of these features and applying an activation function.

In a convolutional layer, we take a "filter" of a fixed size, which can only connect to a small number of neighbouring features at a time. We carry out the weighted sum of this filter at many points in our input data, "sliding" the filter along.

Show the image of the QN architecture

<center><img src="Images/architecture.png" style="height: 300px;"/></center>

#### What is our output?

QuasarNET is trained to identify emission lines via the "line finder" units on the right hand side of this diagram. These work by dividing an input spectrum up into boxes that are equally spaced in log-wavelength:

<center><img src="Images/boxes.png" style="height: 300px;"/></center>

For a spectrum labelled by $i$, the line finder unit for emission line $l$ then is trained to predict two quantities for each box $\alpha$:
1. $\hat{Y}_{il\alpha}$: a number between 0 and 1 to indicate whether emission line $l$ is in box $\alpha$
2. $\hat{X}_{il\alpha}$: a number between 0 and 1 to indicate the offset of the position of line $l$ from the left edge of box $\alpha$

This is easiest to see in an example:

<center><img src="Images/target.png" style="height: 500px;"/></center>

Let's focus on the Ly$\alpha$ line here. We can see that it is in the first box, and hence the first value in the row of $\hat{Y}_{il\alpha}$ corresponding to Ly$\alpha$ is 1, while the others are zero.

Equally, we can see that the Ly$\alpha$ line is most of the way towards the right hand side of this first box. $\hat{X}=0$ would mean that the line sits on the left hand edge, and $\hat{X}=1$ would mean that the line sits on the right hand edge. As such, we'd expect $\hat{X}$ near to 1 for the Ly$\alpha$ line in this example, and indeed we can see a value of 0.96.

These two sets of arrays essentially form our "target" output. 

After we have trained a model, it will then try to predict the values of these arrays for a given spectrum. We 

We construct a loss function from these two arrays, the details of which I won't go into.  

At this stage, we simply need to state which lines we want to look for, and make our loss from the `custom_loss` function we imported at the start.

In [ ]:
lines = ['LYA','CIV(1548)','CIII(1909)', 'MgII(2796)','Hbeta','Halpha']
lines_bal = ['CIV(1548)']

In [ ]:
loss = [custom_loss]*(len(lines)+len(lines_bal))

#### How do we train the model?

There is nothing particularly interesting about the training of QuasarNET models - it follows a fairly standard approach, with no need for anything fancy. 

## QuasarNET at work

### Building a model

Let's get on and build a model.

First we load all the data we need. This is not particularly exciting. The variables `X` contain the spectra, while `Y` contains (coded) information about the object's class and `z` is the VI redshift.

At this stage, we don't have enough data from DESI to train a model, or enough to carry out any substantial tests. As such, we use BOSS DR12 data for now.

In [ ]:
# set nspec to the number of spectra to load. 30000 should be plenty!
nspec = 30000
truth_file=(['/global/cfs/projectdirs/desi/users/jfarr/ML_TensorFlow_tutorial/truth_DR12Q.fits'])
truth = read_truth(truth_file)
tids_full,X_full,Y_full,z_full,bal_full = read_data(['/global/cfs/projectdirs/desi/users/jfarr/ML_TensorFlow_tutorial/data_dr12.fits'], truth, nspec=nspec)

# Load the training data.
data_file = '/global/cfs/projectdirs/desi/users/jfarr/ML_TensorFlow_tutorial/data_train_0.fits'
h = fitsio.FITS(data_file)
tids_train = h[1]['TARGETID'][:]
w = np.in1d(tids_full, tids_train)
X_train = X_full[w]
Y_train = Y_full[w]
z_train = z_full[w]
bal_train = bal_full[w]

# To get the validation data, remove the spectra in the training sample from the full sample
w = ~np.in1d(tids_full, tids_train)
tids_val = tids_full[w]
X_val = X_full[w]
Y_val = Y_full[w]
z_val = z_full[w]
bal_val = bal_full[w]

Now we set up the model ready to carry out training. First we choose how many spectra to use for training, and how many epochs to train for. 10000 spectra and 5 epochs should be plenty.

In [ ]:
ntrain = 10000
nepochs = 5

Now we make the model and our loss function. We imported these from QuasarNET at the start, so it's as simple as just calling a function. We just need to tell the function how may pixels are in our data, and how many lines we're looking to identify.

In [ ]:
model = QuasarNET((X_train.shape[1],1), nlines=len(lines)+len(lines_bal))

Now we may compile our model as before.

In [ ]:
model.compile(optimizer=Adam(), loss=loss, metrics=[])

Finally, we need to generate our "target" function, and then we're ready to go!

In [ ]:
target, sample_weight = objective(z_train[:ntrain],Y_train[:ntrain],bal_train[:ntrain],lines=lines,lines_bal=lines_bal)
loss_history = model.fit(X_train[:ntrain,:,None], target, epochs=nepochs, batch_size=32, sample_weight=sample_weight)

And that's it, our model is trained! When training "real" models to use in actual analyses, we would typically use between 50,000 and 500,000 spectra for training, and may use 100-200 epochs. This takes a few hours on a single compute node at NERSC, perhaps 15 hours at most.

### Assessing our model

Is our lightweight model any good? Let's find out! We use `model.predict` as we did previously to calculate predicted values of our target function. We then apply a processing to this function to make it a bit more easy to interpret.

In [ ]:
p = model.predict(X_val[:,:,None])
c_line, z_line, zbest, c_line_bal, z_line_bal = process_preds(p, lines, lines_bal)

The key output quantities here are:
 - `c_line`: the confidence with which QuasarNET thinks it has identified each emission line in each spectrum
 - `z_line`: the redshift inferred form the position of each emission line

We choose to classify a certain spectrum as a QSO if we have identified at least `ndetect` emission lines with confidence greater than a threshold confidence `c_th`.

#### Individual spectra

Let's look at some spectra and see what classifications and redshifts our QuasarNET model comes up with. We'll set `ndetect` to 1 and `c_th` to 0.5 for now.

In [ ]:
ndetect = 1
c_th = 0.5

In [ ]:
def plot_spectrum(ival,X,c_line,z_line,zbest,Y,z,c_th=0.5,ndetect=1):
    
    fig, ax = plt.subplots(1,1,figsize=(10,6))
    ax.plot(wave, X[ival,:])
    
    isqso_truth = (Y[ival,:].argmax()==2) | (Y[ival,:].argmax()==3)
    isqso_qn = (c_line[:,ival].sum()>c_th)>=ndetect
    
    title = r'Is QSO? VI: {}, QN: {}'.format(isqso_truth,isqso_qn)
    title += '\n'
    title += r'$z_{{VI}}$={:1.3f}, $z_{{QN}}$='.format(z[ival])
    if isqso_qn:
        title += r'{:1.3f}'.format(zbest[ival])
    else:
        title += 'N/A'
    
    ax.set_title(title)
    m = X[ival,:].min()
    M = X[ival,:].max()
    ax.grid()
    ax.set_ylim(m-2,M+2)
    for il,l in enumerate(lines):
        lam = absorber_IGM[l]*(1+z_line[il,ival])
        w = abs(wave-lam)<100
        if w.sum()>0:
            m = X[ival,w].min()-1
            M = X[ival,w].max()+1
            ax.plot([lam,lam], [m,M],'r--', alpha=0.1+0.9*c_line[il,ival])
            ax.text(lam,M+0.5,'c$_{{{}}}={}$'.format(l,round(c_line[il,ival],3)),
                     horizontalalignment='center',alpha=0.1+0.9*c_line[il,ival])
    ax.set_xlabel(r'$\lambda_\mathrm{obs}~[\AA]$')
    ax.set_ylabel(r'renormalised flux')
    plt.show()
    
    return

In [ ]:
plot_spectrum(0,X_val,c_line,z_line,zbest,Y_val,z_val,c_th=c_th,ndetect=ndetect)

In [ ]:
plot_spectrum(10,X_val,c_line,z_line,zbest,Y_val,z_val)

#### Ensemble performance

Let's look at the performance more systematically. There are two quantities that may be of interest here: 

 - Purity (precision) $= \frac{\mathrm{number\ of\ correctly\ classified\ QSOs}}{\mathrm{number\ of\ objects\ classified\ as\ QSOs}}$
 
 - Completeness (recall) $= \frac{\mathrm{number\ of\ correctly\ classified\ QSOs}}{\mathrm{number\ of\ true\ QSOs}}$
 
Here, we count a QSO as being "correctly classified" if QuasarNET is both able to identify it as a QSO, and correctly estimate its redshift to within a velocity error of 6,000 km/s.

We will consider a range of different confidence thresholds between 0 and 1, computing purity and completeness at each point.

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,6))

# Set up our arrays.
c_th = np.arange(0,1,0.01)
pur = c_th*0
com = c_th*0
dv_max = 6000./300000.

# Determine which objects are true QSOs, and which are "bad"
isqso_truth = (Y_val.argmax(axis=1)==2) | (Y_val.argmax(axis=1)==3)
is_bad = Y_val.argmax(axis=1)==4

# Determine which spectra QN accurately estimated the redshift for
zgood = (z_val>0) & (abs(zbest-z_val) < dv_max*(1+z_val))

# For each value of confidence threshold, compute purity and completeness.
for i,cth in enumerate(c_th):
    isqso_qn = (c_line>cth).sum(axis=0)>=ndetect
    ntrue_positives = (isqso_qn & zgood & ~is_bad).sum()
    pur[i] = ntrue_positives/(isqso_qn & (~is_bad)).sum()
    com[i] = (isqso_qn & zgood & isqso_truth).sum()/isqso_truth.sum()
    
# Plot them!
ax.plot(c_th, pur, label='Purity')
ax.plot(c_th, com, label='Completeness')
ax.set_xlim(0.0,1.0)
ax.set_ylim(0.8,1.0)
ax.set_xlabel(r'$c_{th}$')
ax.grid()
ax.legend()

#### Overall impression

We've managed to achieve over 90% purity and completeness with a model that took less than 1 minute to train. Not bad! 

Of course, this is due to clever design of the model, allowing the innate power of neural networks to come to the fore.

### Some DESI data

Last but not least, let's try and classify some actual DESI data. For this task, we'll load some pre-trained results.

These were obtained using a model trained on 10% of the QSO targets from BOSS DR12 data, around 63,000 spectra. This is roughly the same number of QSO target spectra as we'll observe in DESI SV.

In [ ]:
results = fitsio.FITS('/global/cfs/projectdirs/desi/users/jfarr/ML_TensorFlow_tutorial/qn_on_68002.fits')

Let's load some DESI data to classify and redshift. These spectra are from tile 68002, when it was observed on 15th March this year. 

First we load data from the VI team which contains the true classifications and spectra. We only include spectra for which the VI team were very confident.

In [ ]:
truth = read_truth(['/global/cfs/projectdirs/desi/users/jfarr/ML_TensorFlow_tutorial/truth_68002.fits'])
truth = {k: truth[k] for k in truth.keys() if truth[k].z_conf_person==3}

Now the spectra. I've already processed these by smoothing them so we can read them staright from the file.

In [ ]:
# Open the data file and extract the targetids (tids) and spectra (X).
data_file = '/global/cfs/projectdirs/desi/users/jfarr/ML_TensorFlow_tutorial/data_andes_68002_20200315.fits'
h = fitsio.FITS(data_file)
tids = h[1]['TARGETID'][:]
X = h[0][:,:443]
X[:,434:443] = 0.
h.close()

# Include only spectra that we have true classifications/redshifts for, and are in our results.
w = np.array([((t in truth.keys()) & (t in results[1][:]['TARGETID'])) for t in tids])
tids = tids[w]
X = X[w,:]
z = np.array([truth[t].z_vi for t in tids])

We now put our "truth" dictionary into a better form (normally QuasarNET does this automatically, but the main branch cannot yet read DESI data in this way!)

In [ ]:
classes = np.array([truth[t].class_person for t in tids])
z_conf = np.array([truth[t].z_conf_person for t in tids])
Y = np.zeros((X.shape[0],5))

## STAR
w = (classes==1) & (z_conf==3)
Y[w,0] = 1

## GALAXY
w = (classes==4) & (z_conf==3)
Y[w,1] = 1

## QSO_LZ
w = ((classes==3) | (classes==30)) & (z<2.1) & (z_conf==3)
Y[w,2] = 1

## QSO_HZ
w = ((classes==3) | (classes==30)) & (z>=2.1) & (z_conf==3)
Y[w,3] = 1

## BAD
w = z_conf != 3
Y[w,4] = 1

We now extract the results from our fits file into a the format we'd like them in.

In [ ]:
c_line = []
z_line = []
zbest = []
c_line_bal = []
z_line_bal = []
for tid in tids:
    w = np.in1d(results[1][:]['TARGETID'],tid)
    if w.sum()>0:
        c_line += [results[1][:][w]['C_LINES'][0]]
        z_line += [results[1][:][w]['Z_LINES'][0]]
        zbest += [results[1][:][w]['ZBEST'][0]]
        c_line_bal += [results[1][:][w]['C_LINES_BAL'][0]]
        z_line_bal += [results[1][:][w]['C_LINES_BAL'][0]]
c_line = np.array(c_line).T
z_line = np.array(z_line).T
zbest = np.array(zbest)
c_line_bal = np.array(c_line_bal).T
z_line_bal = np.array(z_line_bal).T

In [ ]:
plot_spectrum(0,X,c_line,z_line,zbest,Y,z)

In [ ]:
plot_spectrum(1,X,c_line,z_line,zbest,Y,z)

This seems pretty promising! Let's look at the ensemble performance for these spectra:

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,6))

ndetect = 1

# Set up our arrays.
c_th = np.arange(0,1,0.01)
pur = c_th*0
com = c_th*0
dv_max = 6000./300000.

# Determine which objects are true QSOs, and which are "bad"
isqso_truth = (Y.argmax(axis=1)==2) | (Y.argmax(axis=1)==3)
is_bad = Y.argmax(axis=1)==4

# Determine which spectra QN accurately estimated the redshift for
zgood = (z>0) & (abs(zbest-z) < dv_max*(1+z))

# For each value of confidence threshold, compute purity and completeness.
for i,cth in enumerate(c_th):
    isqso_qn = (c_line>cth).sum(axis=0)>=ndetect
    ntrue_positives = (isqso_qn & zgood & ~is_bad).sum()
    pur[i] = ntrue_positives/(isqso_qn & (~is_bad)).sum()
    com[i] = (isqso_qn & zgood & isqso_truth).sum()/isqso_truth.sum()
    
# Plot them!
ax.plot(c_th, pur, label='Purity')
ax.plot(c_th, com, label='Completeness')
ax.set_xlim(0.0,1.0)
ax.set_ylim(0.8,1.0)
ax.set_xlabel(r'$c_{th}$')
ax.grid()
ax.legend()

Again, looking pretty good, especially given that the model which produced these results had never seen a DESI spectrum, only SDSS ones!

Hopefully this indicates the potential of QuasarNET, and acts as a nice example of the power of TensorFlow.

# BONUS

For anyone interested, below is an example dataset which you can try out some machine learning techniques on. 

Like the very simple linear regression problem previously, it can be solved analytically by carrying out the appropriate matrix calculation. However, it is much more interesting than that toy problem as it contains actual data, and it requires a little bit of thought to apply ML techniques to it.

I've loaded the data below and put it into a some `pandas` DataFrames to make it easier to visualise. If you're not familiar with `pandas`, then don't worry, you can scrap that by just deleting the relevant cell and using the "raw" data from the dictionary `housing` instead.

## The task

In this example, we want to analyse some housing data from California. Our dependent variable is the house price, and the independent variables include the number of rooms in the house, its age etc. The aim is to be able to predict the price of a house given information about its independent variables (i.e. this is a regression problem, not classification).

First we import the data. If you want to know a bit more about the data, then un-comment the last line of the cell below.

In [ ]:
from sklearn.datasets import fetch_california_housing
reset_graph()

housing = fetch_california_housing()
m, n = housing.data.shape

#print(housing['DESCR'])

In [ ]:
housing_data = pd.DataFrame(data=housing['data'], columns=housing['feature_names'])
housing_prices = pd.DataFrame(data=housing['target'], columns=housing['target_names'])

In [ ]:
housing_data

In [ ]:
housing_prices

In order to allow us to have a constant parameter $\theta^{(0)}$, we can add a "bias" term to our housing data. This is just a column of ones.

In [ ]:
bias = pd.Series(data=np.ones((m,)),name='Bias')
housing_data_with_bias = pd.concat([bias,housing_data],axis=1)
housing_data_with_bias

## Analytic solution

As before, we can calculate the analytic solution straightforwardly with:

$$ \hat{\theta} = \left( X^{\rm T} X \right)^{-1} X^{\rm T} y. $$

Below we implement this in TF. Hopefully it makes rough sense at least

In [ ]:
X = tf.constant(housing_data_with_bias, dtype=tf.float32, name="X")
y = tf.constant(housing_prices, dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta_analytic = tf.matmul(tf.matmul(tf.linalg.inv(tf.matmul(XT, X)), XT), y)

In [ ]:
theta_analytic

## Machine learning solutions

You can attack this problem from many angles. Most straightforwardly you can construct a linear regression solution with TF (just this time we're finding the values of 9 parameters rather than 2).

Equally, you could construct a more complex neural network, as we did for the MNIST problem. Feel free to play around and see how things work!

P.S. Don't forget about the "boring" things, like normalising your data etc!